In [1]:
# set up non-QM hardware
import Labber
# connect to server 
client = Labber.connectToServer('localhost') # get list of instruments 
#instruments = client.getListOfInstrumentsString() 
#for instr in instruments: 
#    print(instr) # close connection 

# reset all QDevil channels to 0 V
QDevil = client.connectToInstrument('QDevil QDAC', dict(interface='Serial', address='3'))
for n in range(24):
    if n+1 < 10:
        QDevil.setValue("CH0" + str(n+1) + " Voltage", 0.0)
    else:
        QDevil.setValue("CH" + str(n+1) + " Voltage", 0.0)

DC_ss = -3.651246
QDevil.setValue("CH02 Voltage", DC_ss)

# digital attenuators
Vaunix1 = client.connectToInstrument('Painter Vaunix Lab Brick Digital Attenuator', dict(interface='USB', address='25606'))
Vaunix2 = client.connectToInstrument('Painter Vaunix Lab Brick Digital Attenuator', dict(interface='USB', address='25607'))
ROI = 38
ROO = 10
Vaunix1.setValue("Attenuation", ROI)
Vaunix2.setValue("Attenuation", ROO)

# TWPA pump
SG = client.connectToInstrument('Rohde&Schwarz RF Source', dict(interface='TCPIP', address='192.168.88.2'))
freq_TWPA = 6326E6;
pwr_TWPA = -10;
SG.setValue('Frequency', freq_TWPA)
SG.setValue('Power', pwr_TWPA)

client.close()

In [6]:
# %%
from qm.QuantumMachinesManager import QuantumMachinesManager
from qm.qua import *
from configuration import *
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
%matplotlib qt

# Start to save data
import datetime 
import os
import pandas as pd 

now = datetime.datetime.now()
year = now.strftime("%Y")
month = now.strftime("%m")
day = now.strftime("%d")
hour = now.strftime("%H")
minute = now.strftime("%M")
second  = now.strftime("%S")

tPath = os.path.join(r'Z:\LabberData_DF5\QM_Data_DF5',year,month,'Data_'+month+day)
if not os.path.exists(tPath):
   os.makedirs(tPath)
exp_name = 'time_of_flight'
qubit_name = 'UA_Q2'

f_str =exp_name + '_'+ qubit_name +'_'+ hour+ '_' + minute +'_' + second
f_name= f_str+'.csv'
j_name = f_str + '_state.json'
p_name= f_str+'.png'

# End to save data
#########################################
# Set-up the machine and get the config #
#########################################
machine = QuAM("quam_state.json")

config = build_config(machine)

qubit_index = 5 # LA Q2, the 6th qubit

###################
# The QUA program #
###################
n_avg = 10000  # Number of averaging loops
cooldown_time = 20_000 * u.ns

with program() as raw_trace_prog:
    n = declare(int)
    adc_st = declare_stream(adc_trace=True)

    with for_(n, 0, n < n_avg, n + 1):
        reset_phase(machine.resonators[qubit_index].name)
        measure("readout", machine.resonators[qubit_index].name, adc_st)
        wait(cooldown_time, machine.resonators[qubit_index].name)

    with stream_processing():
        # Will save average:
        adc_st.input1().average().save("adc1")
        adc_st.input2().average().save("adc2")
        # # Will save only last run:
        adc_st.input1().save("adc1_single_run")
        adc_st.input2().save("adc2_single_run")


#####################################
#  Open Communication with the QOP  #
#####################################
qmm = QuantumMachinesManager(machine.network.qop_ip, cluster_name=machine.network.cluster_name, octave=octave_config)

qm = qmm.open_qm(config)
job = qm.execute(raw_trace_prog)
res_handles = job.result_handles
res_handles.wait_for_all_values()
adc1 = u.raw2volts(res_handles.get("adc1").fetch_all())
adc2 = u.raw2volts(res_handles.get("adc2").fetch_all())
adc1_single_run = u.raw2volts(res_handles.get("adc1_single_run").fetch_all())
adc2_single_run = u.raw2volts(res_handles.get("adc2_single_run").fetch_all())

adc1_mean = np.mean(adc1)
adc2_mean = np.mean(adc2)
adc1_unbiased = adc1 - np.mean(adc1)
adc2_unbiased = adc2 - np.mean(adc2)
signal = savgol_filter(np.abs(adc1_unbiased + 1j * adc2_unbiased), 11, 3)
# detect arrival of readout signal
th = (np.mean(signal[:100]) + np.mean(signal[:-100])) / 2
delay = np.where(signal > th)[0][0]
delay = np.round(delay / 4) * 4
dc_offset_i = -adc1_mean
dc_offset_q = -adc2_mean

# Update the config
print(f"DC offset to add to I: {dc_offset_i:.6f} V")
print(f"DC offset to add to Q: {dc_offset_q:.6f} V")
print(f"TOF to add: {delay} ns")

2023-10-04 10:21:51,775 - qm - ERROR    - Encountered connection error from QOP
Traceback (most recent call last):
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\base_api.py", line 27, in wrapped
    return func(*args, **kwargs)
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\frontend_api.py", line 79, in get_version
    response = run_async(self._stub.get_version(Empty(), timeout=self._timeout))
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\utils\async_utils.py", line 37, in run_async
    return create_future(coroutine).result()  # type: ignore[no-any-return]
  File "C:\Users\painter\.conda\envs\labber-qm-env\lib\concurrent\futures\_base.py", line 444, in result
    return self.__get_result()
  File "C:\Users\painter\.conda\envs\labber-qm-env\lib\concurrent\futures\_base.py", line 389, in __get_result
    raise self._exception
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\

In [3]:
# Plot data
fig = plt.figure(figsize=[14, 6])
# plt.rcParams['figure.figsize'] = [12, 8]
plt.subplot(121)
plt.title("Single run")
plt.plot(adc1_single_run, "b", label="I")
plt.plot(adc2_single_run, "r", label="Q")
plt.axhline(y=0.5)
plt.axhline(y=-0.5)
xl = plt.xlim()
yl = plt.ylim()
plt.plot(xl, adc1_mean * np.ones(2), "k--")
plt.plot(xl, adc2_mean * np.ones(2), "k--")
plt.plot(delay * np.ones(2), yl, "k--")
plt.xlabel("Time [ns]")
plt.ylabel("Signal amplitude [V]")
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1),
          fancybox=True, shadow=True, ncol=5)
plt.subplot(122)
plt.title("Averaged run")
plt.plot(adc1, "b", label="I")
plt.plot(adc2, "r", label="Q")
plt.xlabel("Time [ns]")
plt.ylabel("Signal amplitude [V]")
xl = plt.xlim()
yl = plt.ylim()
# plt.axhline(y=0.5)
# plt.axhline(y=-0.5)
plt.plot(xl, adc1_mean * np.ones(2), "k--")
plt.plot(xl, adc2_mean * np.ones(2), "k--")
plt.plot(delay * np.ones(2), yl, "k--")
plt.xlabel("Time [ns]")
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1),
          fancybox=True, shadow=True, ncol=5)
plt.grid("all")
plt.tight_layout(pad=2)
plt.show()

In [4]:
machine.global_parameters.time_of_flight += 12
machine.global_parameters.con1_downconversion_offset_I += 0.008501
machine.global_parameters.con1_downconversion_offset_Q += 0.009905

In [5]:
machine._save("quam_state.json", flat_data=False)

In [ ]:
# plt.savefig(os.path.join(tPath, p_name))

## convert everything to lists so they are easy to parse in Matlab if desired
if isinstance(adc1_single_run,np.ndarray) == True:
   adc1_single_run1=adc1_single_run.tolist()
else:  
    pass
if isinstance(adc2_single_run,np.ndarray) == True:
   adc2_single_run1=adc2_single_run.tolist()
else:  
    pass
if isinstance(adc1,np.ndarray) == True:
   adc11=adc1.tolist()
else:  
    pass
if isinstance(adc2,np.ndarray) == True:
   adc21=adc2.tolist()
else:  
    pass

data = [('Variable', 'Value'), ('adc1_single_run', adc1_single_run1), ('adc2_single_run', adc2_single_run1),('adc1', adc11), ('adc2', adc21)]
df = pd.DataFrame(data,index=None, columns=None)
df.to_csv(os.path.join(tPath, f_name),header=False, index=False)
open(os.path.join(tPath, j_name), "w").write(open("quam_state.json").read())